In [169]:
import datetime
import os
import sys
import pandas as pd
import FinanceDataReader as fdr
from tqdm import tqdm

from tabulate import tabulate
from datetime import timedelta

df_kospi = fdr.StockListing('KOSPI')
df_kosdaq = fdr.StockListing('KOSDAQ')

# 관리종목
df_AdmStock = fdr.StockListing('KRX-ADMIN')

df_krx = pd.concat([df_kospi, df_kosdaq])

# 관리종목 및 우선주 제거(5, 7, 9, K)
df_filterStk = df_krx[~df_krx['Code'].isin(df_AdmStock['Symbol'])]
df_filterStk = df_filterStk[~df_filterStk['Code'].str.endswith(('5', '7', '9', 'K'))]

df_filterStk[0:10]

,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId
0,005930,KR7005930003,삼성전자,KOSPI,,79800,1,1700,2.18,78700,80200,78600,16103311,1283963958700,476388647490000,5969782550,STK
1,000660,KR7000660001,SK하이닉스,KOSPI,,232500,1,9500,4.26,227000,234500,227000,2504323,577496445500,169260549862500,728002365,STK
2,373220,KR7373220003,LG에너지솔루션,KOSPI,,332000,2,-4500,-1.34,340000,343500,331000,161705,54249614000,77688000000000,234000000,STK
3,005380,KR7005380001,현대차,KOSPI,,285500,1,7000,2.51,280500,290000,278500,1486010,425058743500,59788322530500,209416191,STK
4,207940,KR7207940008,삼성바이오로직스,KOSPI,,742000,1,6000,0.82,739000,744000,733000,38750,28654347000,52811108000000,71174000,STK
5,000270,KR7000270009,기아,KOSPI,,131500,1,2400,1.86,129900,133900,129100,2455886,323421203400,52581381835500,399858417,STK
7,068270,KR7068270008,셀트리온,KOSPI,,179700,2,-100,-0.06,179800,181200,178200,376937,67823304000,38993682173100,216993223,STK
8,105560,KR7105560007,KB금융,KOSPI,,77600,2,-700,-0.89,79000,79300,77300,819852,63714960400,31312459187200,403511072,STK
9,005490,KR7005490008,POSCO홀딩스,KOSPI,,368500,2,-2500,-0.67,373500,375500,368500,261738,96947723500,31164498255000,84571230,STK
10,035420,KR7035420009,NAVER,KOSPI,,166900,1,800,0.48,166700,169400,166300,567417,95185911700,27105994338600,162408594,STK


In [192]:
initDate = datetime.date(2018, 12, 28)
bfDate = initDate + timedelta(days=-180)    # 6개월 전 날짜(2018-07-01)

# Create lists to store results
bf_half_year_dates = []
bf_half_year_closes = []
half_year_ratios = []

# 종목 별 6개월전 주가 / 6개월전 시점 / 6개월 수익률 계산
for index, row in tqdm(df_filterStk.iterrows(), total=df_filterStk.shape[0]):
    stkCode = row['Code']
    
    try:
        # 주가 데이터 조회
        df_stockPrice = fdr.DataReader(stkCode, bfDate, initDate)

        # 해당 날짜에 데이터가 있는 경우에만 진행
        if df_stockPrice.shape[0] > 0:
            bfPrice = df_stockPrice.iloc[0]['Close']
            currPrice = df_stockPrice.iloc[-1]['Close']
            
            # Calculate values
            bf_half_year_dates.append(bfDate)
            bf_half_year_closes.append(bfPrice)
            half_year_ratios.append((currPrice - bfPrice) / bfPrice * 100)
        else:
            # 해당 시점에 주가 없으면 None 처리
            bf_half_year_dates.append(None)
            bf_half_year_closes.append(None)
            half_year_ratios.append(None)
    except Exception as e:
        print(f'{e.args}')
        bf_half_year_dates.append(None)
        bf_half_year_closes.append(None)
        half_year_ratios.append(None)

df_tmp = pd.DataFrame({
    'Code': df_filterStk['Code'],
    'BF_HALF_YEAR_DATE': bf_half_year_dates,
    'BF_HALF_YEAR_CLOSE': bf_half_year_closes,
    'HALF_YEAR_RATIO': half_year_ratios
})

# 디스플레이 욥션
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

100%|██████████| 2477/2477 [02:29<00:00, 16.55it/s]


In [193]:
# 기존 종목 리스트에 6개월 데이터 추가
df_tmp1 = pd.concat([df_filterStk, df_tmp.drop(columns=['Code'])], axis=1)

In [195]:
# 수익률 기준으로 오름차순 정렬
df_tmp1 = df_tmp1.dropna(subset=['BF_HALF_YEAR_DATE'])

df_tmp2 = df_tmp1.sort_values(by='HALF_YEAR_RATIO', ascending=False)
df_tmp2

df_result = df_tmp2[0:20]


,Code,ISU_CD,Name,Market,Dept,Close,ChangeCode,Changes,ChagesRatio,Open,High,Low,Volume,Amount,Marcap,Stocks,MarketId,BF_HALF_YEAR_DATE,BF_HALF_YEAR_CLOSE,HALF_YEAR_RATIO
803,051630,KR7051630002,진양화학,KOSPI,,3580,2,-15,-0.42,3670,3670,3530,7309,26156610,56564000000,15800000,STK,2018-07-01,1939.0,199.948427
827,009460,KR7009460007,한창제지,KOSPI,,807,2,-3,-0.37,810,814,803,21119,17077409,48151661202,59667486,STK,2018-07-01,996.0,198.192771
926,238090,KR7238090005,앤디포스,KOSDAQ,벤처기업부,3700,2,-215,-5.49,3795,3810,3590,103465,381305180,87102821100,23541303,KSQ,2018-07-01,4939.0,194.270095
513,143160,KR7143160000,아이디스,KOSDAQ,우량기업부,15590,2,-310,-1.95,15910,16010,15380,42384,661240700,167068520100,10716390,KSQ,2018-07-01,7210.0,193.342580
400,002320,KR7002320000,한진,KOSPI,,20000,1,360,1.83,19660,20300,19520,54972,1095609080,298952560000,14947628,STK,2018-07-01,19093.0,168.333944
470,008350,KR7008350001,남선알미늄,KOSPI,,1791,2,-14,-0.78,1804,1805,1790,326344,585976618,231180650190,129079090,STK,2018-07-01,1110.0,154.954955
142,025980,KR7025980004,아난티,KOSDAQ,우량기업부,5670,2,-70,-1.22,5730,5750,5660,232542,1324890000,502529140260,88629478,KSQ,2018-07-01,7600.0,146.710526
1011,950110,KR8392070007,SBI핀테크솔루션즈,KOSDAQ,외국기업(소속부없음),3240,1,60,1.89,3115,3395,3115,2830,9155450,77930229600,24052540,KSQ,2018-07-01,6870.0,137.263464
1237,263920,KR7263920001,휴엠앤씨,KOSDAQ,벤처기업부,1161,2,-10,-0.85,1164,1185,1154,77387,90060964,56941400574,49045134,KSQ,2018-07-01,4179.0,134.290500
745,000890,KR7000890004,보해양조,KOSPI,,508,2,-7,-1.36,528,530,505,2440900,1251987921,70673025532,139120129,STK,2018-07-01,850.0,122.352941


In [206]:
# 2018-07-01   2018-12-28
test = fdr.DataReader('238090', '2018-07-01', '2018-12-28')

# 펀더멘털 데이터 조회하기
test2 = fdr.SnapDataReader(f'NAVER/FINSTATE/005930')
test2

주요재무정보,매출액,영업이익,영업이익(발표기준),세전계속사업이익,당기순이익,당기순이익(지배),당기순이익(비지배),자산총계,부채총계,자본총계,자본총계(지배),자본총계(비지배),자본금,영업활동현금흐름,투자활동현금흐름,재무활동현금흐름,CAPEX,FCF,이자발생부채,영업이익률,순이익률,ROE(%),ROA(%),부채비율,자본유보율,EPS(원),PER(배),BPS(원),PBR(배),현금DPS(원),현금배당수익률,현금배당성향(%),발행주식수(보통주)
날짜,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2019-12-01,2304009.0,277685.0,277685.0,304322.0,217389.0,215051.0,2338.0,3525645.0,896841.0,2628804.0,2549155.0,79649.0,8975.0,453829.0,-399482.0,-94845.0,253678.0,200152.0,184120.0,12.05,9.44,8.69,6.28,34.12,28856.02,3166.0,17.63,37528.0,1.49,1416.0,2.54,44.73,5.969783e+09
2020-12-01,2368070.0,359939.0,359939.0,363451.0,264078.0,260908.0,3170.0,3782357.0,1022877.0,2759480.0,2676703.0,82777.0,8975.0,652870.0,-536286.0,-83278.0,375920.0,276950.0,202174.0,15.20,11.15,9.98,7.23,37.07,30692.79,3841.0,21.09,39406.0,2.06,2994.0,3.70,77.95,5.969783e+09
2021-12-01,2796048.0,516339.0,516339.0,533518.0,399074.0,392438.0,6637.0,4266212.0,1217212.0,3048999.0,2962377.0,86622.0,8975.0,651054.0,-330478.0,-239910.0,471221.0,179833.0,183921.0,18.47,14.27,13.92,9.92,39.92,33143.62,5777.0,13.55,43611.0,1.80,1444.0,1.84,25.00,5.969783e+09
2022-12-01,3022314.0,433766.0,433766.0,464405.0,556541.0,547300.0,9241.0,4484245.0,936749.0,3547496.0,3451861.0,95635.0,8975.0,621813.0,-316028.0,-193900.0,494304.0,127509.0,103332.0,14.35,18.41,17.07,12.72,26.41,38144.29,8057.0,6.86,50817.0,1.09,1444.0,2.61,17.92,5.969783e+09
2023-12-01,2589355.0,65670.0,65670.0,110063.0,154871.0,144734.0,10137.0,4559060.0,922281.0,3636779.0,3532338.0,104441.0,8975.0,441374.0,-169228.0,-85931.0,576113.0,-134739.0,126859.0,2.54,5.98,4.15,3.43,25.36,39114.28,2131.0,36.84,52002.0,1.51,1444.0,1.84,67.78,5.969783e+09
2024-12-01,3089363.0,390404.0,NaN,430180.0,360875.0,345939.0,NaN,4910304.0,1005586.0,3904719.0,3796888.0,NaN,8979.0,719213.0,-602080.0,-80238.0,527777.0,176222.0,NaN,12.64,11.68,9.44,7.62,25.75,NaN,5093.0,15.34,55897.0,1.40,1505.0,1.93,25.97,NaN
2025-12-01,3427612.0,551322.0,NaN,590439.0,479535.0,460112.0,NaN,5321120.0,1041725.0,4279394.0,4159311.0,NaN,8979.0,840957.0,-604937.0,-93966.0,551907.0,275594.0,NaN,16.09,13.99,11.57,9.37,24.34,NaN,6774.0,11.53,61232.0,1.28,1441.0,1.85,18.70,NaN
2026-12-01,3698967.0,603626.0,NaN,652038.0,542920.0,520004.0,NaN,5824558.0,1070378.0,4754179.0,4619622.0,NaN,8978.0,926681.0,-607271.0,-86503.0,554588.0,340809.0,NaN,16.32,14.68,11.85,9.74,22.51,NaN,7655.0,10.20,68009.0,1.15,1461.0,1.87,16.77,NaN
